## Load required packages

In [1]:
using DataFrames
using CSV
using Latexify
using Pipe
using ZipFile

Setting for printing

In [2]:
ENV["COLUMNS"]=500
ENV["LINES"]= 40

40

## Load data

In [3]:
z = ZipFile.Reader("../data/data-for-capture-recapture.csv.zip") 
a_file_in_zip = filter(x -> x.name == "data-for-capture-recapture.csv", z.files)[1]
cr_data = CSV.read(a_file_in_zip, DataFrame) 
first(cr_data, 10)

,id_unit,id_ad,date_start,date_end,date_arch,vacancies,q1,q2,q3,q4,q1_days,q2_days,q3_days,q4_days,woj,pkd,size,sector,source
,String,String,Date…,Date…,Date…?,Int64,Bool,Bool,Bool,Bool,Int64?,Int64?,Int64?,Int64?,Int64,String,String,Int64,String
1,1b4701354603e4f186a443940768aecf1e38d65c,b88aaaf7d35782005f90dc1c9d2dc0dc27902285,2018-03-21,2018-04-03,2018-04-04,1,1,0,0,0,10,missing,missing,missing,8,R,S,1,cbop
2,32f40ce1c624214de376f791a6e0e365c5cdeea6,da840976961bcf05d9b670fbf9063a8e9e76277c,2018-11-21,2019-01-07,2019-01-07,1,0,0,0,1,missing,missing,missing,40,12,A,D,1,cbop
3,623fa6eee4d0e31cf1db41724d4630ae35f8cec0,ed5193dabf23fe46eeee8df4aa532ef6c958831a,2018-03-15,2018-04-11,2018-04-11,1,1,0,0,0,16,missing,missing,missing,30,P,D,1,cbop
4,623fa6eee4d0e31cf1db41724d4630ae35f8cec0,282a00d492521b2f522d5e54cc0e4d9ee885e2cd,2018-03-16,2018-04-13,2018-04-10,1,1,0,0,0,15,missing,missing,missing,30,P,D,1,cbop
5,623fa6eee4d0e31cf1db41724d4630ae35f8cec0,7cd7a9ec93f21cdc3028f6ae67c49e484a47d8a1,2018-03-12,2018-04-30,2018-04-23,1,1,0,0,0,19,missing,missing,missing,30,P,D,1,cbop
6,dd6e32263037e81f124a256869cf48ea319c0e22,3319f8670628fc65269b4450df964c260a1e3d63,2018-03-28,2018-04-15,2018-04-15,1,1,0,0,0,3,missing,missing,missing,2,P,D,1,cbop
7,dd6e32263037e81f124a256869cf48ea319c0e22,dfda7d3be12c076c80b897e9b0831d2f6005c543,2018-03-28,2018-04-15,2018-04-15,1,1,0,0,0,3,missing,missing,missing,2,P,D,1,cbop
8,d0d15c10127d71d32a3b95ef50bb039cbe1c2293,675f6a2198d30f6529b577000e73f61178d96109,2018-06-26,2018-07-25,2018-07-25,1,0,1,0,0,missing,4,missing,missing,22,P,D,1,cbop
9,d0d15c10127d71d32a3b95ef50bb039cbe1c2293,ce0a2b7a25685691885dc8582ec05be61f7e2712,2018-06-26,2018-07-25,2018-07-25,1,0,1,0,0,missing,4,missing,missing,22,P,D,1,cbop


### Statistics for results

Statistics for table `Distribution of auxiliary variables in sampling frame and units identified in non-probability sources (pooled data)`

In [5]:
@pipe cr_data |> 
    unique(_, [:id_unit, :source, :sector]) |>
    groupby(_, [:source, :sector]) |> 
    combine(_, nrow => :count) |> 
    transform(_, :count => (x -> round.(x/1000, digits =1)) => :count) |>
    groupby(_, [:source]) |>
    transform(_, :count => sum => :total) |>
    transform(_, [:count, :total] => ByRow((x,y) -> round(x/y*100, digits = 1)) => :p) |>
    select(_,Not(:total)) 

,source,sector,count,p
,String,Int64,Float64,Float64
1,cbop,1,4.7,11.8
2,cbop,2,35.2,88.2
3,pracuj,1,0.6,3.7
4,pracuj,2,15.8,96.3


In [6]:
## d-> big, s-> medium, 
@pipe cr_data |> 
    unique(_, [:id_unit, :source, :size]) |>
    groupby(_, [:source, :size]) |> 
    combine(_, nrow => :count) |> 
    transform(_, :count => (x -> round.(x/1000, digits =1)) => :count) |>
    groupby(_, [:source]) |>
    transform(_, :count => sum => :total) |>
    transform(_, [:count, :total] => ByRow((x,y) -> round(x/y*100, digits = 1)) => :p) |>
    select(_,Not(:total)) |> 
    sort(_, [:source, :size])

,source,size,count,p
,String,String,Float64,Float64
1,cbop,D,8.4,21.1
2,cbop,M,18.1,45.4
3,cbop,S,13.4,33.6
4,pracuj,D,7.0,42.7
5,pracuj,M,4.0,24.4
6,pracuj,S,5.4,32.9


In [7]:
@pipe cr_data |> 
    unique(_, [:id_unit, :source, :pkd]) |>
    groupby(_, [:source, :pkd]) |> 
    combine(_, nrow => :count) |> 
    transform(_, :count => (x -> round.(x/1000, digits =1)) => :count) |>
    groupby(_, [:source]) |>
    transform(_, :count => sum => :total) |>
    transform(_, [:count, :total] => ByRow((x,y) -> round(x/y*100, digits = 1)) => :p) |>
    select(_,Not(:total)) |> 
    sort(_, [:source, :pkd]) 

,source,pkd,count,p
,String,String,Float64,Float64
1,cbop,A,0.4,1.0
2,cbop,B,0.1,0.3
3,cbop,C,8.9,22.4
4,cbop,D,0.1,0.3
5,cbop,E,0.6,1.5
6,cbop,F,5.5,13.8
7,cbop,G,8.8,22.1
8,cbop,H,2.2,5.5
9,cbop,I,2.5,6.3


Number records by days before end of quarter

In [8]:
@pipe cr_data |>
    select(_, :id_ad, :source, r"days") |>
    stack(_, r"days") |>
    dropmissing(_) |>
    transform(_, :value => ByRow( x -> x == 0 ? "zero" :
                                       x <= 10 ? "to 10" :
                                       (x > 10 && x <= 20) ? "(10,20]" :
                                       (x > 20 && x <= 30) ? "(20,30]" :
                                       (x > 30 && x <= 60) ? "(30,60]" :
                                       (x > 60 && x <= 365) ? "(60,365]" :
                                       "over year") => :value_n) |>
    groupby(_, [:source, :value_n]) |>
    combine(_, nrow => :count) |>
    unstack(_, :value_n, :source, :count) |> 
    latexify(_; env=:table, latex=false,adjustment = :r)

L"\begin{tabular}{rrr}
value_n & cbop & pracuj\\
to 10 & 24553 & 33507\\
(10,20] & 23102 & 54081\\
(20,30] & 14873 & 47455\\
(30,60] & 14434 & 118\\
(60,365] & 11041 & missing\\
over year & 61 & missing\\
zero & 341 & 690\\
\end{tabular}
"

## Table for each quarter

In [9]:
function create_data(data, quarter::Symbol, quarter_days::Symbol, days::Int)
    out_df = filter(quarter => ==(true), data)
    out_df = filter(quarter_days => <=(days), out_df)
    out_df = unique(out_df[:,[:source, :id_unit, :size]])
    out_df[!, :n] .= 1
    out_df = unstack(out_df, [:id_unit, :size], :source, :n)
    out_df.cbop = coalesce.(out_df.cbop, 0)
    out_df.pracuj = coalesce.(out_df.pracuj, 0)
    out_df[!, :size2] .= ifelse.(out_df.size .== "D", "D", "SM")
    out_df = groupby(out_df, [:size2, :cbop, :pracuj])
    out_df = combine(out_df, nrow => :count)
    sort!(out_df, [:size2, :cbop, :pracuj], rev = true) 
    return(out_df)
end

create_data (generic function with 1 method)

In [10]:
@pipe vcat(
    create_data(cr_data, :q1, :q1_days, 30),
    create_data(cr_data, :q2, :q2_days, 30), 
    create_data(cr_data, :q3, :q3_days, 30), 
    create_data(cr_data, :q4, :q4_days, 30),     
    source = :quarter
) |> 
    unstack(_, [:size2, :cbop, :pracuj], :quarter, :count) |> 
    latexify(_; env=:table, latex=false, adjustment = :r)

L"\begin{tabular}{rrrrrrr}
size2 & cbop & pracuj & 1 & 2 & 3 & 4\\
SM & 1 & 1 & 100 & 129 & 107 & 76\\
SM & 1 & 0 & 8900 & 8571 & 8199 & 4019\\
SM & 0 & 1 & 3641 & 3543 & 3116 & 2795\\
D & 1 & 1 & 534 & 582 & 552 & 303\\
D & 1 & 0 & 2584 & 2705 & 2657 & 1528\\
D & 0 & 1 & 3780 & 3608 & 3506 & 3202\\
\end{tabular}
"

In [11]:
@pipe vcat(
    create_data(cr_data, :q1, :q1_days, 30),
    create_data(cr_data, :q2, :q2_days, 30), 
    create_data(cr_data, :q3, :q3_days, 30), 
    create_data(cr_data, :q4, :q4_days, 30),     
    source = :quarter
) |>
    groupby(_,[:quarter,:size2]) |>
    combine(_, :count=> sum =>:total) |>
    unstack(_, :size2, :quarter, :total) |> 
    latexify(_; env=:table, latex=false, adjustment = :r) 

L"\begin{tabular}{rrrrr}
size2 & 1 & 2 & 3 & 4\\
SM & 12641 & 12243 & 11422 & 6890\\
D & 6898 & 6895 & 6715 & 5033\\
\end{tabular}
"

Estimated $\hat{c}$

In [12]:
@pipe vcat(
    create_data(cr_data, :q1, :q1_days, 30),
    create_data(cr_data, :q2, :q2_days, 30), 
    create_data(cr_data, :q3, :q3_days, 30), 
    create_data(cr_data, :q4, :q4_days, 30),     
    source = :quarter
) |> 
    filter(:cbop => ==(1), _) |>
    select(_, Not(:cbop)) |> 
    unstack(_, [:quarter, :size2], :pracuj, :count) |>
    sort(_,[:size2, :quarter]) |>
    transform(_, ["1", "0"] => ByRow((x, y) -> x/(x+y)) => :c) |>
    select(_, [:size2, :quarter, :c]) |>
    unstack(_, :size2, :quarter, :c) |> 
    latexify(_; env=:table, latex=false, adjustment = :r, fmt = "%.4f")

L"\begin{tabular}{rrrrr}
size2 & 1 & 2 & 3 & 4\\
D & 0.1713 & 0.1771 & 0.1720 & 0.1655\\
SM & 0.0111 & 0.0148 & 0.0129 & 0.0186\\
\end{tabular}
"